In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import sys
sys.path.append('models') # append the directory containing the module

In [247]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import EncoderEEG
import deepcnn
import cnn2dlstm
import cnn1dlstm
import traintest
import LSTM_CNN_Encoder
import warnings
warnings.filterwarnings('ignore')

In [251]:
import importlib
importlib.reload(traintest) # reload the module

<module 'traintest' from 'C:\\Users\\CSE IIT BHILAI\\Desktop\\ML_Project-main\\models\\traintest.py'>

In [189]:
torch.cuda.is_available()
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

GPU Name: NVIDIA GeForce RTX 3060


In [17]:
dataset=torch.load(r"C:\Users\CSE IIT BHILAI\Downloads\eeg_signals_raw_with_mean_std.pth")
labels=dataset['labels']
print(labels)

['n02389026', 'n03888257', 'n03584829', 'n02607072', 'n03297495', 'n03063599', 'n03792782', 'n04086273', 'n02510455', 'n11939491', 'n02951358', 'n02281787', 'n02106662', 'n04120489', 'n03590841', 'n02992529', 'n03445777', 'n03180011', 'n02906734', 'n07873807', 'n03773504', 'n02492035', 'n03982430', 'n03709823', 'n03100240', 'n03376595', 'n03877472', 'n03775071', 'n03272010', 'n04069434', 'n03452741', 'n03792972', 'n07753592', 'n13054560', 'n03197337', 'n02504458', 'n02690373', 'n03272562', 'n04044716', 'n02124075']


In [19]:
data=dataset['dataset']

In [23]:
splits=torch.load(r"C:\Users\CSE IIT BHILAI\Downloads\block_splits_by_image_all.pth")

In [25]:
x_train,x_val,x_test,y_train,y_val,y_test=traintest.Splitter(data,splits,subject_no=0)

In [27]:
# For using 2D CNN or similar models, we need to add an extra dimension to our data for which we can use unsqueeze function

x_train=torch.stack(x_train)
#x_train=x_train.unsqueeze(3)
x_val=torch.stack(x_val)
#x_val=x_val.unsqueeze(3)
x_test=torch.stack(x_test)
#x_test=x_test.unsqueeze(3)
y_train=torch.tensor(y_train)
y_val=torch.tensor(y_val)
y_test=torch.tensor(y_test)

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val,y_val)
test_dataset = TensorDataset(x_test,y_test)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=16,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=16,shuffle=True)

In [239]:
num_classes = 40
learning_rate = 0.005
num_epochs = 50

model = EncoderEEG.EEGEncoder(num_classes=40).to(device)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

EEGEncoder(
  (temporal_block): TemporalBlock(
    (network): Sequential(
      (0): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(3,))
      (1): ReLU()
      (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(6,), dilation=(2,))
      (3): ReLU()
      (4): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(12,), dilation=(4,))
      (5): ReLU()
      (6): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(24,), dilation=(8,))
      (7): ReLU()
    )
  )
  (spatial_block): SpatialBlock(
    (network): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (res_blocks): ModuleList(
    (0-1): 2 x ResidualBlock(
      (conv1)

In [ ]:
#for inputs,labels in train_loader:
 #   print(inputs.shape)

In [241]:
# Call training function
traintest.train_model(model, train_loader, val_loader,criterion,optimizer,device,num_epochs)

Epoch 1/50, Training Loss: 3.4523, Validation Loss: 3.1078, Training Accuracy: 4.37%, Validation Accuracy: 7.71%
Epoch 2/50, Training Loss: 2.9296, Validation Loss: 2.9162, Training Accuracy: 8.19%, Validation Accuracy: 10.06%
Epoch 3/50, Training Loss: 2.6252, Validation Loss: 2.5055, Training Accuracy: 11.08%, Validation Accuracy: 13.26%
Epoch 4/50, Training Loss: 2.5202, Validation Loss: 2.3933, Training Accuracy: 12.50%, Validation Accuracy: 14.51%
Epoch 5/50, Training Loss: 2.4074, Validation Loss: 2.2381, Training Accuracy: 14.27%, Validation Accuracy: 14.81%
Epoch 6/50, Training Loss: 2.2777, Validation Loss: 2.1278, Training Accuracy: 16.80%, Validation Accuracy: 18.57%
Epoch 7/50, Training Loss: 2.1516, Validation Loss: 1.9069, Training Accuracy: 19.50%, Validation Accuracy: 26.38%
Epoch 8/50, Training Loss: 2.0808, Validation Loss: 2.0793, Training Accuracy: 20.31%, Validation Accuracy: 18.92%
Epoch 9/50, Training Loss: 2.1114, Validation Loss: 1.8864, Training Accuracy: 20.9

In [253]:
traintest.evaluate_model(model,device,test_loader)

Test Accuracy: 72.96%


After about 50 Epochs, Our Model shows a decent enough accuracy in the range of 70-75%, now let's train it for 10 more epochs, but first, we will save our model.

In [267]:
#Saving the model

# Define the path where you want to save the model
model_save_path = 'model.pth'

# Save model and optimizer states
torch.save({
    'model_state_dict': model.state_dict(),  # Model weights
    'optimizer_state_dict': optimizer.state_dict(),  # Optimizer state                       # Optionally save loss or other metrics
}, model_save_path)

Loading the Model to resume training

In [281]:
checkpoint = torch.load('model.pth')

# Load the model and optimizer state dictionaries
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [283]:
traintest.train_model(model, train_loader, val_loader,criterion,optimizer,device,10)

Epoch 1/10, Training Loss: 0.2664, Validation Loss: 0.2590, Training Accuracy: 91.74%, Validation Accuracy: 91.24%
Epoch 2/10, Training Loss: 0.2164, Validation Loss: 0.2487, Training Accuracy: 93.31%, Validation Accuracy: 91.59%
Epoch 3/10, Training Loss: 0.1752, Validation Loss: 0.2297, Training Accuracy: 94.84%, Validation Accuracy: 92.94%
Epoch 4/10, Training Loss: 0.1662, Validation Loss: 0.2266, Training Accuracy: 94.54%, Validation Accuracy: 92.19%
Epoch 5/10, Training Loss: 0.1668, Validation Loss: 0.2657, Training Accuracy: 94.60%, Validation Accuracy: 91.29%
Epoch 6/10, Training Loss: 0.1429, Validation Loss: 0.2178, Training Accuracy: 95.88%, Validation Accuracy: 92.09%
Epoch 7/10, Training Loss: 0.1352, Validation Loss: 0.1704, Training Accuracy: 96.04%, Validation Accuracy: 94.79%
Epoch 8/10, Training Loss: 0.1371, Validation Loss: 0.2617, Training Accuracy: 95.45%, Validation Accuracy: 90.84%
Epoch 9/10, Training Loss: 0.1190, Validation Loss: 0.1858, Training Accuracy: 9

In [285]:
traintest.evaluate_model(model,device,test_loader)

Test Accuracy: 93.84%


In [287]:
torch.save(model.state_dict(), 'EEGEncoder_state_dict.pth')  
torch.save(model, 'EEGEncoder.pth')  
torch.save(optimizer.state_dict(), 'optimizer_state_dict.pth')

Saving the Final Model with an accuracy of 93%

## Not Relevant

In [303]:
sample_input = torch.randn(16, 128, 440)  # Create a sample input tensor

# Initialize your model
  # Adjust based on the number of classes in your task
model =EncoderEEG.EEGEncoder(num_classes=40)  # Use the correct model name here

# Pass the sample input through the model
try:
    output, latent = model(sample_input)
    print("Output shape:", output.shape)  # Print the output shape
    print(output)
except Exception as e:
    print("An error occurred:", e)

Output shape: torch.Size([16, 40])
tensor([[-0.1648, -0.0817, -0.1377,  0.3391, -0.0531,  0.0718,  0.2142,  0.0264,
          0.2497,  0.0196, -0.2046, -0.0046, -0.1044,  0.1499, -0.2323, -0.0176,
         -0.0105,  0.1582,  0.1340, -0.1335,  0.2519,  0.1530,  0.2221, -0.3653,
          0.0656,  0.1462, -0.1331,  0.0540,  0.2554, -0.0571, -0.2028, -0.1892,
          0.0844, -0.2205,  0.0201,  0.1821, -0.3183,  0.1828,  0.1488,  0.0144],
        [-0.1652, -0.0849, -0.1379,  0.3438, -0.0526,  0.0759,  0.2167,  0.0274,
          0.2525,  0.0213, -0.2072, -0.0060, -0.1050,  0.1521, -0.2345, -0.0188,
         -0.0102,  0.1596,  0.1354, -0.1387,  0.2540,  0.1548,  0.2242, -0.3691,
          0.0646,  0.1492, -0.1351,  0.0552,  0.2581, -0.0568, -0.2085, -0.1919,
          0.0838, -0.2200,  0.0185,  0.1820, -0.3188,  0.1844,  0.1483,  0.0147],
        [-0.1631, -0.0808, -0.1372,  0.3382, -0.0522,  0.0710,  0.2150,  0.0255,
          0.2483,  0.0188, -0.2040, -0.0062, -0.1033,  0.1475, -0.2319, 

In [305]:
print(model.parameters)

<bound method Module.parameters of EEGEncoder(
  (temporal_block): TemporalBlock(
    (network): Sequential(
      (0): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(3,))
      (1): ReLU()
      (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(6,), dilation=(2,))
      (3): ReLU()
      (4): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(12,), dilation=(4,))
      (5): ReLU()
      (6): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(24,), dilation=(8,))
      (7): ReLU()
    )
  )
  (spatial_block): SpatialBlock(
    (network): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (res_blocks): ModuleList(
    (0-1